In [ ]:
# 3. Imports
import cv2
import time
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tqdm import tqdm

from transformers import AutoImageProcessor, AutoModelForSemanticSegmentation
import torch
import cv2
import time

In [ ]:
# Carregar modelo pré-treinado de segmentação semântica chamado deeplabv3_mobilenet_v2_1.0_513

processor = AutoImageProcessor.from_pretrained("google/deeplabv3_mobilenet_v2_1.0_513")
model = AutoModelForSemanticSegmentation.from_pretrained("google/deeplabv3_mobilenet_v2_1.0_513")

# Essa função pega um frame de vídeo (em formato NumPy RGB) 
# retorna um mapa de segmentação, onde cada pixel tem um valor correspondente à classe detectada (ex: rua, pessoa, carro...)

def segment_frame_hf(np_img):
    inputs = processor(images=np_img, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    seg = processor.post_process_semantic_segmentation(outputs, target_sizes=[np_img.shape[:2]])[0]
    return np.array(seg)

In [ ]:
# Aplicar segmentação semântica em 1 frame a cada 5 segundos, salvando o tempo de processamento de cada frame.


video_path = '/content/drive/MyDrive/Atividade 4 - Visão Computacional/cidade'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("\nErro ao abrir o vídeo.\n")
else:
    print("\nVídeo carregado com sucesso.\n")
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Total de frames: {total_frames}")  # Total de frames no vídeo
    print(f"FPS (frames por segundo): {fps}")  # Quantos frames existem por segundo

# Processar a cada 5 segundos
step = int(fps * 5)
results = []

print(f"\n🟦 Processando 1 frame a cada 5 segundos (step = {step} frames)")
steps_segundos = [1, 2, 5]
resultados_dict = {}

for step_segundos in steps_segundos:
    print(f"\n🟦 Processando 1 frame a cada {step_segundos} segundos")
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    step = int(fps * step_segundos)

    resultados = []

    for i in tqdm(range(0, total_frames, step), desc=f"{step_segundos}s"):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        print(f"Processando frame {i} - Leitura OK? {ret}")
        if not ret:
            continue

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        start = time.time()
        seg = segment_frame_hf(rgb_frame)
        end = time.time()

        tempo = end - start
        resultados.append(tempo)
        print(f"Frame {i} processado em {tempo:.2f}s")

    resultados_dict[step_segundos] = resultados
    cap.release()




In [ ]:

print("Total de frames:", total_frames)
print("FPS:", fps)
print("Step:", step)
print("Chaves disponíveis em resultados_dict:", resultados_dict.keys())


In [ ]:
# Gráfico comparando o tempo médio de processamento em diferentes taxas
media_por_step = [np.mean(resultados_dict[s]) for s in steps_segundos]

plt.figure(figsize=(8, 5))
plt.plot(steps_segundos, media_por_step, marker='o', linestyle='-')
for x, y in zip(steps_segundos, media_por_step):
    plt.text(x, y + 0.02, f"{y:.2f}s", ha='center')
plt.xlabel("Intervalo de captura (segundos)")
plt.ylabel("Tempo médio de processamento (s)")
plt.title("Impacto da taxa de amostragem no tempo de processamento")
plt.grid(True)
plt.xticks(steps_segundos)
plt.show()


In [ ]:

cap = cv2.VideoCapture(video_path)
frame_indices = list(range(0, total_frames, step))[:3] # Lista com os índices dos frames a serem exibidos (primeiros 3 frames processados)

plt.figure(figsize=(15, 5))

for idx, i in enumerate(frame_indices):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    seg = segment_frame_hf(rgb)
    seg_color = cv2.applyColorMap((seg * 20).astype(np.uint8), cv2.COLORMAP_JET)
    seg_color = cv2.resize(seg_color, (frame.shape[1], frame.shape[0]))
    overlay = cv2.addWeighted(rgb, 0.6, seg_color, 0.4, 0)

    plt.subplot(1, 3, idx + 1)
    plt.imshow(overlay)
    plt.title(f'Frame {i}')
    plt.axis('off')

plt.suptitle("Exemplos de segmentação")
plt.show()


In [ ]:
output_path = '/content/drive/MyDrive/trab/Theodor_segmentado.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 

fps = 5 # velocidade de reprodução

cap = cv2.VideoCapture(video_path)

# Ler o primeiro frame para pegar altura e largura
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, frame = cap.read()
if not ret:
    raise ValueError("\nNão foi possível ler o primeiro frame do vídeo.")

height, width, _ = frame.shape
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Processar o vídeo completo
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
for i in range(0, total_frames, step):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    seg = segment_frame_hf(rgb)
    
    # Converter a segmentação para cores
    seg_color = cv2.applyColorMap((seg * 20).astype(np.uint8), cv2.COLORMAP_JET)
    seg_color = cv2.resize(seg_color, (frame.shape[1], frame.shape[0]))

    # Sobrepor a segmentação na imagem original
    overlay = cv2.addWeighted(frame, 0.6, seg_color, 0.4, 0)
    out.write(overlay)

out.release()
print(f"\n Novo vídeo salvo em: {output_path}")
